# ADD-ON Dataset


**tez_lib**

https://www.kaggle.com/abhishek/tez-lib

**longformerlarge4096**

https://www.kaggle.com/hengzheng/longformerlarge4096

In [ ]:
import sys
sys.path.append("/kaggle/input/tez-lib/")
import tez

In [ ]:
import gc
gc.enable()

import os

import numpy as np
import pandas as pd
import tez
import torch
import torch.nn as nn
from joblib import Parallel, delayed
from transformers import AutoConfig, AutoModel, AutoTokenizer

# Configuration

In [ ]:
class config:
    is_amp=True
    discourse_marker_to_label = {
    "B-Lead": 0,
    "I-Lead": 1,
    "B-Position": 2,
    "I-Position": 3,
    "B-Evidence": 4,
    "I-Evidence": 5,
    "B-Claim": 6,
    "I-Claim": 7,
    "B-Concluding Statement": 8,
    "I-Concluding Statement": 9,
    "B-Counterclaim": 10,
    "I-Counterclaim": 11,
    "B-Rebuttal": 12,
    "I-Rebuttal": 13,
    "O": 14,
    "PAD": -100,
    }
    label_to_discourse_marker = {}
    num_discourse_marker = 15
    
target_id_map = {
    "B-Lead": 0,
    "I-Lead": 1,
    "B-Position": 2,
    "I-Position": 3,
    "B-Evidence": 4,
    "I-Evidence": 5,
    "B-Claim": 6,
    "I-Claim": 7,
    "B-Concluding Statement": 8,
    "I-Concluding Statement": 9,
    "B-Counterclaim": 10,
    "I-Counterclaim": 11,
    "B-Rebuttal": 12,
    "I-Rebuttal": 13,
    "O": 14,
    "PAD": -100,
}

id_target_map = {v: k for k, v in target_id_map.items()}

# https://www.kaggle.com/kausheeki01/feedback-prize-2022-pytorch-better-parameters
proba_thresh = {
    "Lead": 0.687,
    "Position": 0.537,
    "Evidence": 0.637,
    "Claim": 0.537,
    "Concluding Statement": 0.687,
    "Counterclaim": 0.537,
    "Rebuttal": 0.537,
}
# To remove the data which is less than 2% in word length
# https://www.kaggle.com/phanttan/ner-longformer
min_thresh = {
    "Lead": 9,
    "Position": 5,
    "Evidence": 14,
    "Claim": 3,
    "Concluding Statement": 11,
    "Counterclaim": 6,
    "Rebuttal": 4,
}
class args1:
    input_path = "../input/feedback-prize-2021/"
    model = "../input/longformerlarge4096/longformer-large-4096/"
    tez_model= "../input/fblongformerlarge1536/"
    output = "."
    batch_size = 8
    max_len = 4096
    
class args2:
    input_path = "../input/feedback-prize-2021/"
    model = "../input/longformerlarge4096/longformer-large-4096/"
    tez_model= "../input/tez-fb-large/"
    output = "."
    batch_size = 8
    max_len = 4096

# Helper Functions

In [ ]:
def time_to_str(t, mode='min'):
    # t in second
    if mode=='min':
        t = int(t)/60
        minute = t%60
        hr = t//60
        return f'%2d hr %02d minute'%(hr, minute)
    elif mode=='second':
        second = int(t)
        minute = second//60
        return f'%2d min %02d sec'%(min,sec)
    else:
        raise NotImplementedError

In [ ]:
class FeedbackDataset:
    def __init__(self, samples, max_len, tokenizer):
        self.samples = samples
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.length = len(samples)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        input_ids = self.samples[idx]["input_ids"]

        # add start token id to the input_ids
        input_ids = [self.tokenizer.cls_token_id] + input_ids

        if len(input_ids) > self.max_len - 1:
            input_ids = input_ids[: self.max_len - 1]

        # add end token id to the input_ids
        input_ids = input_ids + [self.tokenizer.sep_token_id]
        attention_mask = [1] * len(input_ids)

        return {
            "ids": input_ids,
            "mask": attention_mask,
        }

## Collate Function

In [ ]:
# https://www.kaggle.com/bacicnikola/sequence-bucketing-pytorch-implementation
class Collate:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, batch):
        output = dict()
        output["ids"] = [sample["ids"] for sample in batch]
        output["mask"] = [sample["mask"] for sample in batch]

        # calculate max token length of this batch
        batch_max = max([len(ids) for ids in output["ids"]])

        # add padding
        if self.tokenizer.padding_side == "right":
            output["ids"] = [s + (batch_max - len(s)) * [self.tokenizer.pad_token_id] for s in output["ids"]]
            output["mask"] = [s + (batch_max - len(s)) * [0] for s in output["mask"]]
        else:
            output["ids"] = [(batch_max - len(s)) * [self.tokenizer.pad_token_id] + s for s in output["ids"]]
            output["mask"] = [(batch_max - len(s)) * [0] + s for s in output["mask"]]

        # convert to tensors
        output["ids"] = torch.tensor(output["ids"], dtype=torch.long)
        output["mask"] = torch.tensor(output["mask"], dtype=torch.long)

        return output

In [ ]:
# https://www.kaggle.com/kausheeki01/feedback-prize-2022-pytorch-better-parameters
class FeedbackModel(tez.Model):
    def __init__(self, model_name, num_labels):
        super().__init__()
        self.model_name = model_name
        self.num_labels = num_labels
        config = AutoConfig.from_pretrained(model_name)

        hidden_dropout_prob: float = 0.22
        layer_norm_eps: float = 17589e-7
        config.update(
            {
                "output_hidden_states": True,
                "hidden_dropout_prob": hidden_dropout_prob,
                "layer_norm_eps": layer_norm_eps,
                "add_pooling_layer": False,
            }
        )
        self.transformer = AutoModel.from_config(config)
        self.output = nn.Linear(config.hidden_size, self.num_labels)

    def forward(self, ids, mask):
        transformer_out = self.transformer(ids, mask)
        sequence_output = transformer_out.last_hidden_state
        logits = self.output(sequence_output)
        logits = torch.softmax(logits, dim=-1)
        return logits, 0, {}

In [ ]:
def word_probability_to_predict_df(text_to_word_probability, id):
    return

In [ ]:
def do_threshold(submit_df, use=['length', 'probability']):
    return

In [ ]:
# https://www.kaggle.com/kaggleqrdl/tensorflow-longformer-ner-postprocessing
def jn(pst, start, end):
    return " ".join([str(x) for x in pst[start:end]])
def link_evidence(oof):
    thresh = 1
    idu = oof['id'].unique()
    idc = idu[1]
    eoof = oof[oof['class'] == "Evidence"]
    neoof = oof[oof['class'] != "Evidence"]
    thresh2 = 26 # Need the explaination
    retval = []
    for idv in idu:
        for c in  ['Lead', 'Position', 'Evidence', 'Claim', 'Concluding Statement',
               'Counterclaim', 'Rebuttal']:
            q = eoof[(eoof['id'] == idv) & (eoof['class'] == c)]
            if len(q) == 0:
                continue
            pst = []
            for i,r in q.iterrows():
                pst = pst +[-1] + [int(x) for x in r['predictionstring'].split()]
            start = 1
            end = 1
            for i in range(2,len(pst)):
                cur = pst[i]
                end = i
                #if pst[start] == 205:
                #   print(cur, pst[start], cur - pst[start])
                if (cur == -1 and c != 'Evidence') or ((cur == -1) and ((pst[i+1] > pst[end-1] + thresh) or (pst[i+1] - pst[start] > thresh2))):
                    retval.append((idv, c, jn(pst, start, end)))
                    start = i + 1
            v = (idv, c, jn(pst, start, end+1))
            retval.append(v)
    roof = pd.DataFrame(retval, columns = ['id', 'class', 'predictionstring']) 
    roof = roof.merge(neoof, how='outer')
    return roof

In [ ]:
def compute_overlap(predict, truth):
    """
    Calculates the overlap between prediction and
    ground truth and overlap percentages used for determining
    true positives.
    """
    # Length of each and intersection
    try:
        len_truth   = len(truth)
        len_predict = len(predict)
        intersect = len(truth & predict)
        overlap1 = intersect/ len_truth
        overlap2 = intersect/ len_predict
        return (overlap1, overlap2)
    except:  # at least one of the input is NaN
        return (0, 0)

def compute_f1_score_one(predict_df, truth_df, discourse_type):
    """
    A function that scores for the kaggle
        Student Writing Competition

    Uses the steps in the evaluation page here:
        https://www.kaggle.com/c/feedback-prize-2021/overview/evaluation
    """
    t_df = truth_df.loc[truth_df['discourse_type'] == discourse_type,   ['id', 'predictionstring']].reset_index(drop=True)
    p_df = predict_df.loc[predict_df['class'] == discourse_type,  ['id', 'predictionstring']].reset_index(drop=True)

    p_df.loc[:,'predict_id'] = p_df.index
    t_df.loc[:,'truth_id'] = t_df.index
    p_df.loc[:,'predictionstring'] = [set(p.split(' ')) for p in p_df['predictionstring']]
    t_df.loc[:,'predictionstring'] = [set(p.split(' ')) for p in t_df['predictionstring']]

    # Step 1. all ground truths and predictions for a given class are compared.
    joined = p_df.merge(t_df,
                           left_on='id',
                           right_on='id',
                           how='outer',
                           suffixes=('_p','_t')
                          )
    overlap = [compute_overlap(*predictionstring) for predictionstring in zip(joined.predictionstring_p, joined.predictionstring_t)]

    # 2. If the overlap between the ground truth and prediction is >= 0.5,
    # and the overlap between the prediction and the ground truth >= 0.5,
    # the prediction is a match and considered a true positive.
    # If multiple matches exist, the match with the highest pair of overlaps is taken.
    joined['potential_TP'] = [(o[0] >= 0.5 and o[1] >= 0.5) for o in overlap]
    joined['max_overlap' ] = [max(*o) for o in overlap]
    joined_tp = joined.query('potential_TP').reset_index(drop=True)
    tp_pred_ids = joined_tp\
        .sort_values('max_overlap', ascending=False) \
        .groupby(['id','truth_id'])['predict_id'].first()

    # 3. Any unmatched ground truths are false negatives
    # and any unmatched predictions are false positives.
    fp_pred_ids = set(joined['predict_id'].unique()) - set(tp_pred_ids)

    matched_gt_ids   = joined_tp['truth_id'].unique()
    unmatched_gt_ids = set(joined['truth_id'].unique()) -  set(matched_gt_ids)

    # Get numbers of each type
    TP = len(tp_pred_ids)
    FP = len(fp_pred_ids)
    FN = len(unmatched_gt_ids)
    f1 = TP / (TP + 0.5*(FP+FN))
    return f1

def compute_lb_f1_score(predict_df, truth_df):
    f1_score = {}
    for discourse_type in truth_df.discourse_type.unique():
        f1_score[discourse_type] = compute_f1_score_one(predict_df, truth_df, discourse_type)
    #f1 = np.mean([v for v in class_scores.values()])
    return f1_score

## prepare_test_data

In [ ]:
def _prepare_test_data_helper(args, tokenizer, ids):
    test_samples = []
    for idx in ids:
        filename = os.path.join(args.input_path, "test", idx + ".txt")
        with open(filename, "r") as f:
            text = f.read()

        encoded_text = tokenizer.encode_plus(
            text,
            add_special_tokens=False,
            return_offsets_mapping=True,
        )
        input_ids = encoded_text["input_ids"]
        offset_mapping = encoded_text["offset_mapping"]

        sample = {
            "id": idx,
            "input_ids": input_ids,
            "text": text,
            "offset_mapping": offset_mapping,
        }

        test_samples.append(sample)
    return test_samples


def prepare_test_data(df, tokenizer, args):
    test_samples = []
    ids = df["id"].unique()
    ids_splits = np.array_split(ids, 4)

    results = Parallel(n_jobs=4, backend="multiprocessing")(
        delayed(_prepare_test_data_helper)(args, tokenizer, idx) for idx in ids_splits
    )
    for result in results:
        test_samples.extend(result)

    return test_samples

# Inferring

In [ ]:
df_test = pd.read_csv(os.path.join("../input/feedback-prize-2021/", "sample_submission.csv"))
df_ids = df_test["id"].unique()

tokenizer = AutoTokenizer.from_pretrained(args1.model)
test_samples = prepare_test_data(df_test, tokenizer, args1)
collate = Collate(tokenizer=tokenizer)

In [ ]:
raw_preds = []
for fold_ in range(10):
    current_idx = 0
    test_dataset = FeedbackDataset(test_samples, args1.max_len, tokenizer)
    
    if fold_ < 5:
        model = FeedbackModel(model_name=args1.model, num_labels=len(target_id_map) - 1)
        model.load(os.path.join(args1.tez_model, f"model_{fold_}.bin"), weights_only=True)
        preds_iter = model.predict(test_dataset, batch_size=args1.batch_size, n_jobs=-1, collate_fn=collate)
    else:
        model = FeedbackModel(model_name=args2.model, num_labels=len(target_id_map) - 1)
        model.load(os.path.join(args2.tez_model, f"model_{fold_-5}.bin"), weights_only=True)
        preds_iter = model.predict(test_dataset, batch_size=args2.batch_size, n_jobs=-1, collate_fn=collate)
        
    current_idx = 0
    
    for preds in preds_iter:
        preds = preds.astype(np.float16)
        preds = preds / 10
        if fold_ == 0:
            raw_preds.append(preds)
        else:
            raw_preds[current_idx] += preds
            current_idx += 1
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
final_preds = []
final_scores = []

for rp in raw_preds:
    pred_class = np.argmax(rp, axis=2)
    pred_scrs = np.max(rp, axis=2)
    for pred, pred_scr in zip(pred_class, pred_scrs):
        pred = pred.tolist()
        pred_scr = pred_scr.tolist()
        final_preds.append(pred)
        final_scores.append(pred_scr)

for j in range(len(test_samples)):
    tt = [id_target_map[p] for p in final_preds[j][1:]]
    tt_score = final_scores[j][1:]
    test_samples[j]["preds"] = tt
    test_samples[j]["pred_scores"] = tt_score

# Submission

In [ ]:
submissions = []
for sample_idx, sample in enumerate(test_samples):
    preds = sample['preds']
    offset_mapping = sample["offset_mapping"]
    sample_id = sample["id"]
    sample_text = sample["text"]
    sample_input_ids = sample["input_ids"]
    sample_pred_scores = sample["pred_scores"]
    sample_preds = []

    if len(preds) < len(offset_mapping):
        preds = preds + ["O"] * (len(offset_mapping) - len(preds))
        sample_pred_scores = sample_pred_scores + [0] * (len(offset_mapping) - len(sample_pred_scores))
    
    idx = 0
    phrase_preds = []
    while idx < len(offset_mapping):
        start, _ = offset_mapping[idx]
        if preds[idx] != "O":
            label = preds[idx][2:]
        else:
            label = "O"
        phrase_scores = []
        phrase_scores.append(sample_pred_scores[idx])
        idx += 1
        while idx < len(offset_mapping):
            if label == "O":
                matching_label = "O"
            else:
                matching_label = f"I-{label}"
            if preds[idx] == matching_label:
                _, end = offset_mapping[idx]
                phrase_scores.append(sample_pred_scores[idx])
                idx += 1
            else:
                break
        if "end" in locals():
            phrase = sample_text[start:end]
            phrase_preds.append((phrase, start, end, label, phrase_scores))
    temp_df = []
    for phrase_idx, (phrase, start, end, label, phrase_scores) in enumerate(phrase_preds):
        word_start = len(sample_text[:start].split())
        word_end = word_start + len(sample_text[start:end].split())
        word_end = min(word_end, len(sample_text.split()))
        ps = " ".join([str(x) for x in range(word_start, word_end)])
        if label != "O":
            if sum(phrase_scores) / len(phrase_scores) >= proba_thresh[label]:
                if len(ps.split()) >= min_thresh[label]:
                    temp_df.append((sample_id, label, ps))
    
    temp_df = pd.DataFrame(temp_df, columns=["id", "class", "predictionstring"])
    submissions.append(temp_df)

submissions = pd.concat(submissions).reset_index(drop=True)
submissions = link_evidence(submissions)
submissions.to_csv("submission.csv", index=False)

In [ ]:
submissions.head()

# Reference

https://www.kaggle.com/librauee/infer-fast-ensemble-models/

https://www.kaggle.com/kausheeki01/feedback-prize-2022-pytorch-better-parameters/notebook